In [1]:
import requests
import re
import time
import string
import random
import math
import pandas as pd
from urllib.parse import urlparse
from collections import Counter
from bs4 import BeautifulSoup
from pymystem3 import Mystem

# Задание №1

In [2]:
def crawler(urls):
    links_tree = set()
    for url in urls:
        if len(texts) >= 101:
            return
    
        if url in visited:
            continue
        visited.add(url)
        
        try:
            #получаем контент страницы и декодируем его
            html = requests.get(url).content.decode()
        except:
            time.sleep(1)
            continue
        
        #парсинг страницы
        soup = BeautifulSoup(html)
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text(' ').strip().replace('\xa0', ' ').replace('\n', '').replace('\r', '').replace('\t', '')
        text = re.sub(' +', ' ', text)
    
        if len(text.split(' ')) >= 101:
            texts[url] = text
            
        parsed = urlparse(url)
        base = f"{parsed.scheme}://{parsed.netloc}"    
        links = re.findall('''<a\s+(?:[^>]*?\s+)?href="([^"]*)"''', html) 
        
        for i, link in enumerate(links):    
            if not urlparse(link).netloc:    
                link_with_base = base + link    
                links[i] = link_with_base
                
        links_tree.update(set(filter(lambda x: 'mailto' not in x, links)))
    
    crawler(links_tree)

In [3]:
texts = {}
visited = set()
start = 'https://m.habr.com/ru/all/'

crawler([start])

In [4]:
index = {i:k for i, k in enumerate(texts)} 

In [5]:
with open('index.txt', 'w') as f:
    for i in index:
        f.write(f"{i}: {index[i]}\n")

In [6]:
!cat index.txt

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


# Задание №2

In [7]:
from openpyxl import load_workbook
import pandas as pd
import os, pymorphy2

In [52]:
words_all = {}
lemma_words = {}
lemma_normal_forms = {}

In [63]:
def get_normal_forms(word_list):
    # получить список нормальных форм
    morph, lst = pymorphy2.MorphAnalyzer(), []
    return [morph.parse(word)[0].normal_form
            for word in word_list]

In [64]:
def get_sparsed_list(word_list):
    # получить список всех словоформ
    morph, lst = pymorphy2.MorphAnalyzer(), []
    for word in word_list:
        lemma = ""
        for x in morph.parse(word)[0].lexeme:
            lemma+=x.word + " "
        lst.append(f"{word} {lemma}\n")
    return lst

In [65]:
texts_lemmatized = {}

for url in texts:
    text = texts[url]
    # оставляем только слова
    text = re.sub(r'[^a-zA-Zа-яА-Я]+', ' ', text)
   
    words = text.split(' ')
    words_all[url]=words
    
    lemma_words[url]=get_sparsed_list(words_all[url])  
    lemma_normal_forms[url]=get_normal_forms(words_all[url])  

In [56]:
with open('words.txt', 'w') as f:
    for url in words_all:
        for i, score in enumerate(words_all[url]):
            f.write(f"{score}\n")

In [43]:
with open('lemma_words.txt', 'w') as f:
    for url in lemma_words:
        for i, score in enumerate(lemma_words[url]):
            f.write(f"{score}\n")

# Задание №3

In [66]:
docs = [index[k] for k in index]

In [67]:
reverse_index = {}

for url in lemma_normal_forms:
    for word in lemma_normal_forms[url]:
        if not word in reverse_index:
            reverse_index[word] = set([url])
        else:
            reverse_index[word].update([url])

In [68]:
logic = {
    '&': lambda x,y: x & y,
    '|': lambda x,y: x | y,
    '!': lambda x: set([doc for doc in docs if doc not in x])
}

def find(query):
    query = query.replace('(', ' ( ').replace(')', ' ) ').replace('!','! ').split()
    
    indexes_words = [i for i,x in enumerate(query) if not x in ['!', '&', '|']]
    for i in indexes_words:
        query[i] = reverse_index[query[i]]
    
    indexes_not = [i for i,x in enumerate(query) if x == '!']
    for i in indexes_not:
        x = query[i+1]
        query[i+1] = logic['!'](x)
        query.pop(i)
    
    indexes_and = [i for i,x in enumerate(query) if x == '&']
    for i in indexes_and:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['&'](x,y)
        query.pop(i)
        query.pop(i)
    
    indexes_or = [i for i,x in enumerate(query) if x == '|']
    for i in indexes_or:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['|'](x,y)
        query.pop(i)
        query.pop(i)
    
    return query[0]

In [69]:
x1 = random.choice(list(reverse_index.keys()))
x2 = random.choice(list(reverse_index.keys()))
x3 = random.choice(list(reverse_index.keys()))

In [70]:
query = f'{x1} | {x2} & !{x3}'
print(query)

полоска | снижать & !itnews


In [71]:
find(query)

{'https://m.habr.com/ru/company/oleg-bunin/blog/543386/',
 'https://m.habr.com/ru/company/ruvds/profile/',
 'https://m.habr.com/ru/post/409639/'}